In [ ]:
#Strats:
#random, 
import os
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
import pydrake.systems.framework as framework
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
    Simulator,
    StartMeshcat,
    DiagramBuilder,
    SpatialForce,
    AddMultibodyPlantSceneGraph,
)
import numpy as np
from random import random
from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario
from pydrake.multibody.plant import  ExternallyAppliedSpatialForce

In [ ]:
ball_model = "/work/ball.sdf"
pitch_model = "/work/pitch.sdf"
goal_model = "/work/Soccergoal.sdf"
attacker_model = "/work/attack_robots_.sdf" 
defender_model = "/work/defend_robots_.sdf"

In [ ]:
MASS_BALL = 1.1454717
GRAVITY = 9.81
MASS_PLAYER = 1.728014228187
PASSING_MULTIPLIER = 1
SHOOTING_MULTIPLIER = 1.8
NET_DIMS = [4.5, 3]

In [ ]:
def create_senario_data():
    scenario_data = f"""
    directives:
    - add_model:
        name: pitch
        file: file://{pitch_model}
    - add_weld:
        parent: world
        child: pitch::pitch_center  
    - add_model:
        name: ball
        file: file://{ball_model}
        default_free_body_pose:
            ball: # Change here!
                translation: [0, 20, .2]
                rotation: !Rpy {{ deg: [42, 33, 18] }}  
    - add_model:
        name: goal
        file: file://{goal_model}
        default_free_body_pose:
            Soccergoal:
                translation: [0., 0., 0.]
                rotation: !Rpy {{ deg: [90, 0, 180] }} 
    """
    player_locs = [[0,20], [-5,17], [5,17]]
    for i,p in enumerate(player_locs):
        location_x = p[0]+ (np.random.rand()*4 -2)
        location_y = p[1]+ (np.random.rand()*4 -2)
        name = "attacker"+ str(i)
        scenario_data += f"""
    - add_model:
        name: {name}
        file: file://{attacker_model}
        default_free_body_pose:
            attack_robots_:
                translation: [{location_x}, {location_y}, 1.5]
                rotation: !Rpy {{ deg: [90, 0, 180] }}"""

    defender_locs = [[0,5], [-3,6], [3,6]]
    for i, d in enumerate(defender_locs):
        location_x = d[0]+ (np.random.rand()*2 -2)
        location_y = d[1]+ (np.random.rand()*2 -2)
        name = "defender"+ str(i)
        scenario_data += f"""
    - add_model:
        name: {name}
        file: file://{defender_model}
        default_free_body_pose:
            police_robots_:
                translation: [{location_x}, {location_y}, 1.5]
                rotation: !Rpy {{ deg: [90, 0, 0] }}"""
                
    return scenario_data

In [ ]:
TIME = 0
meshcat = StartMeshcat()
scenario = load_scenario(data=create_senario_data())
station = MakeHardwareStation(scenario, meshcat)
plant = station.GetSubsystemByName("plant")
simulator = Simulator(station)
context = simulator.get_mutable_context()
station_context = station.GetMyMutableContextFromRoot(context)
plant_context = plant.GetMyMutableContextFromRoot(context)

INFO:drake:Meshcat listening for connections at https://443541ce-e170-4459-8340-438ed0837988.deepnoteproject.com/7000/


Both `d` and `Tr` parameters defined for "07___Default". Use the value of `d` for dissolve (line 20 in .mtl.)


In [ ]:
def check_goal(plant, plant_context):
    X_MAX = 10
    pos = plant.GetPositions(
    plant_context, plant.GetModelInstanceByName("ball")
    )[4:]
    X = (-1*NET_DIMS[0]/2, NET_DIMS[0]/2)
    Y = (0, 1)
    Z = (0, NET_DIMS[1])

    x_condition = X[0] <= pos[0] <= X[1]
    y_condition = Y[0] <= pos[1] <= Y[1]
    z_condition = Z[0] <= pos[2] <= Z[1]

    goal = x_condition and y_condition and z_condition

    if goal:
        return True, True 
    else:
        if pos[1] < -2 or pos[0] < -1*X_MAX or pos[0] > X_MAX :
            return True , False
        else:
            return False,False 

In [ ]:
def get_forces(plant, plant_context, force_vector, name, MULTIPLIER= 1):
    # Create the SpatialForce object
    spatial_force = SpatialForce(tau=np.zeros(3), f=np.array(force_vector)*MULTIPLIER)
    body_index = plant.GetBodyIndices(plant.GetModelInstanceByName(name))[0]
    force_object = ExternallyAppliedSpatialForce()
    force_object.body_index = body_index
    force_object.F_Bq_W = spatial_force
    return force_object

In [ ]:
def apply_forces(plant, plant_context, forces):
    plant.get_applied_spatial_force_input_port().FixValue(plant_context, forces)

In [ ]:
def remove_velocities(plant, plant_context, name):
    #set all velocities to 0
    vel = plant.GetVelocities(plant_context, plant.GetModelInstanceByName(name))
    plant.SetVelocities(plant_context, plant.GetModelInstanceByName(name), np.zeros_like(vel))

In [ ]:
def set_position(plant, plant_context, location, name):
    plant.SetPositions(plant_context, plant.GetModelInstanceByName(name), location)

In [ ]:
def clean_players(plant, plant_context, num_attackers = 3, num_defenders = 3):
    objects = [str(f'defender{i}') for i in range(num_defenders)] + [str(f'attacker{i}') for i in range(num_attackers)]
    forces =[]
    for obj in objects:
        remove_velocities(plant, plant_context,obj )
        forces.append(get_forces(plant, plant_context, [0,0,GRAVITY*MASS_PLAYER], obj))
    return forces

In [ ]:
def shoot_ball(plant, plant_context, sim_time_step, WIND = .5):
    ball_pos = plant.GetPositions(
    plant_context, plant.GetModelInstanceByName("ball")
    ) 
    vel = plant.GetVelocities(plant_context, plant.GetModelInstanceByName("ball")
    ) [:3]

    set_position(plant, plant_context, ball_pos, "ball")
    goal_target = [np.random.rand()*NET_DIMS[0]/4,0,NET_DIMS[1]/2*np.random.rand()]
    time_to_travel =np.random.rand() +.5 

    displacement = np.array(goal_target) - np.array(ball_pos[4:])
    avg_velocity_needed = displacement / time_to_travel
    acceleration_needed = (avg_velocity_needed - vel) / time_to_travel
    force = (MASS_BALL * acceleration_needed)/sim_time_step

    wind_force =WIND* np.random.normal()

    
    return get_forces(plant, plant_context, force+wind_force, "ball",MULTIPLIER=SHOOTING_MULTIPLIER)

In [ ]:
def pass_ball(plant_context,plant,player_index,sim_time_step, num_attacker=3):
    pass_to_person = player_index
    while pass_to_person == player_index:
        pass_to_person = np.random.randint(0,num_attacker)

    next_pos = plant.GetPositions(
    plant_context, plant.GetModelInstanceByName("attacker"+str(pass_to_person))
    )[4:]
    
    pos = plant.GetPositions(
    plant_context, plant.GetModelInstanceByName("ball")
    ) [4:]
    vel = plant.GetVelocities(plant_context, plant.GetModelInstanceByName("ball")
    ) [:3]

    time_to_travel =.5+ np.random.rand() 

    displacement = np.array(next_pos) - np.array(pos)
    avg_velocity_needed = displacement / time_to_travel
    acceleration_needed = (avg_velocity_needed -vel) / time_to_travel
    force = (MASS_BALL * acceleration_needed)/sim_time_step

    
    return get_forces(plant, plant_context, force, "ball", MULTIPLIER= PASSING_MULTIPLIER), pass_to_person, force

In [ ]:
def move_defenders(plant, plant_context, sim_time_step, num_defenders=3, defender_movement_distance=0.5):
    """
    Move defenders in the simulation, applying random jitter to their positions.

    Args:
        plant (Plant): The plant model.
        plant_context (Context): The context of the plant.
        sim_time_step (float): The simulation time step.
        num_defenders (int): Number of defenders.
        defender_movement_distance (float): Movement distance for defenders.

    Returns:
        None: This function modifies the defenders' positions in-place.
    """
    for i in range(num_defenders):
        defender_name = "defender" + str(i)
        defender_pos = plant.GetPositions(plant_context, plant.GetModelInstanceByName(defender_name))

        random_jitter = np.random.normal(0, 1, 3) * defender_movement_distance
        random_jitter[2] = 0  # Assuming the z-axis is vertical and should not change

        defender_pos_new = defender_pos[4:] + random_jitter * sim_time_step
        defender_pos[4:] = defender_pos_new

        set_position(plant, plant_context, defender_pos, defender_name)


In [ ]:
def move_attackers(plant, plant_context, current_attacker_index, sim_time_step, num_attackers=3,
                   attacker_movement_distance=3, collect=False):
    """
    Move attackers towards the goal and handle ball movement.
    
    Args:
        plant (Plant): The plant model.
        plant_context (Context): The context of the plant.
        current_attacker_index (int): Index of the current attacker.
        sim_time_step (float): The simulation time step.
        num_attackers (int): Number of attackers.
        attacker_movement_distance (float): Movement distance for attackers.
        collect (bool): Flag to determine movement strategy.

    Returns:
        np.array: New position of the ball.
    """
    attacker_with_ball = "attacker" + str(current_attacker_index)
    ball_pos = plant.GetPositions(plant_context, plant.GetModelInstanceByName("ball"))

    goal_target = np.array([0, 0, 0])
    direction_to_goal = goal_target - ball_pos[4:]
    direction_norm = np.linalg.norm(direction_to_goal)
    direction_to_goal_normalized = direction_to_goal / direction_norm if direction_norm != 0 else np.zeros_like(direction_to_goal)
    
    COLLECT_MOVEMENT_DISTANCE = 3
    if collect:  # If collect is true, reduce movement distance
        attacker_movement_distance =.5

    for i in range(num_attackers):
        attacker_name = "attacker" + str(i)
        attacker_pos = plant.GetPositions(plant_context, plant.GetModelInstanceByName(attacker_name))
        random_jitter = np.random.normal(0, 1, 3) * attacker_movement_distance / 3
        random_jitter[2] = 0  # Assuming z-axis is vertical and should not change

        attacker_pos_new = attacker_pos[4:] + (direction_to_goal_normalized * attacker_movement_distance + random_jitter) * sim_time_step
        attacker_pos[4:] = attacker_pos_new
        set_position(plant, plant_context, attacker_pos, attacker_name)

        if attacker_name == attacker_with_ball:
            if collect:
                direction_to_ball = ball_pos[4:] - attacker_pos[4:]
                direction_to_ball[2] = 0
                direction_norm_ball = np.linalg.norm(direction_to_ball)
                direction_to_ball_normalized = direction_to_ball / direction_norm_ball if direction_norm_ball != 0 else np.zeros_like(direction_to_ball)

                attacker_pos_new = attacker_pos[4:] + (direction_to_ball_normalized * COLLECT_MOVEMENT_DISTANCE ) * sim_time_step
                attacker_pos[4:] = attacker_pos_new
                set_position(plant, plant_context, attacker_pos, attacker_name)
                collected =  ( np.linalg.norm(attacker_pos_new - ball_pos[4:]) <1.5)
            else:
                radius = 0.5
                ball_jitter = np.random.normal(0, 1, 3)
                ball_jitter[2] = 0
                ball_jitter_norm = ball_jitter / np.linalg.norm(ball_jitter) * radius
                ball_pos_new = attacker_pos_new + ball_jitter_norm
                ball_pos_new[2] = .3
                ball_pos[4:] = ball_pos_new
                set_position(plant, plant_context, ball_pos, "ball")

    if collect:
        return ball_pos[4:] , collected
    return ball_pos_new, None

In [ ]:
def next_state(ball_loc):
    shooting_zone = [5, 13]
    
    if ball_loc[1] >=shooting_zone[0] and ball_loc[1] <=shooting_zone[1]:
        if (np.random.rand() >=.85):
            return "shooting"
        elif (np.random.rand() >=.85):
            return "passing"
        else:
            return "moving"
    elif ball_loc[1] <shooting_zone[1]:
        return "shooting"
    else:
        if (np.random.rand() >=.95):
            return "passing"
        else:
            return "moving"

In [ ]:
def move_ball(plant, plant_context, current_attacker_index, sim_time_step):
    move_defenders(plant, plant_context, sim_time_step)
    ball_loc, _ = move_attackers(plant, plant_context,current_attacker_index, sim_time_step)
    return next_state(ball_loc)

In [ ]:
def collect_ball(plant, plant_context, current_attacker_index, sim_time_step):
    move_defenders(plant, plant_context, sim_time_step)
    ball_loc, collected= move_attackers(plant, plant_context,current_attacker_index, sim_time_step,collect=True)
    if not collected:
        return "collecting" 
        
    return next_state(ball_loc)

In [ ]:
def reset_episode(plant, plant_context):
    attacker_locs = [[0,25], [-5,22], [5,22]]
    defender_locs = [[0,5], [-3,6], [3,6]]

    for i,p in enumerate(attacker_locs):
        location_x = p[0]+ (np.random.rand()*2 -1)
        location_y = p[1]+ (np.random.rand()*2 -1)
        name = "attacker"+ str(i)
        attacker_pos = plant.GetPositions(plant_context, plant.GetModelInstanceByName(name))
        attacker_pos[4:] = np.array([location_x, location_y, 1 ])
        set_position(plant, plant_context, attacker_pos, name)
        if i == 0:
            ball_pos = plant.GetPositions(plant_context, plant.GetModelInstanceByName("ball"))
            ball_pos[4:] = np.array([location_x, location_y+1, 1.5] )
            set_position(plant, plant_context, ball_pos, "ball")
    
    for i,p in enumerate(defender_locs):
        location_x = p[0]+ (np.random.rand()*1 -1)
        location_y = p[1]+ (np.random.rand()*1 -1)
        name = "defender"+ str(i)
        defender_pos = plant.GetPositions(plant_context, plant.GetModelInstanceByName(name))
        defender_pos[4:] = np.array([location_x, location_y, 1] )
        set_position(plant, plant_context, defender_pos, name)

In [ ]:
def run_episode(plant, plant_context, TIME, sim_time_step = .1, max_sim = 20 ):
    reset_episode(plant, plant_context)
    meshcat.StartRecording()
    time_ = 0
    current_attacker_index = 0
    #states shooting, moving, passing, collecting_ball, waiting
    state = "moving"
    forces = []
    shot_time = 0
    MAX_TIME_SINCE_SHOT = 5 
    passing_force= None
    while True:
        #Step
        apply_forces(plant, plant_context, forces)
        time_ += sim_time_step
        simulator.AdvanceTo(time_ +TIME)
        forces = clean_players(plant,plant_context)

        if state == "moving":
            #to accelerte increase sim_time_step in moving/collecting/waiting
            state = move_ball(plant, plant_context, current_attacker_index, sim_time_step)
            if passing_force is not None:
                #apply counter force to ball
                forces.append(get_forces(plant, plant_context, -1*passing_force, "ball",MULTIPLIER= PASSING_MULTIPLIER))
                passing_force = None
        elif state == "passing":
            force, current_attacker_index, passing_force = pass_ball(plant_context,plant,current_attacker_index,sim_time_step)
            forces.append(force)
            state = "collecting"
        elif state == "shooting":
            force = shoot_ball(plant, plant_context, sim_time_step)
            forces.append(force)
            state = "waiting"
        elif state == "collecting":
            state = collect_ball(plant, plant_context, current_attacker_index, sim_time_step)
        elif state == "waiting":
            move_defenders(plant, plant_context, sim_time_step)
            shot_time+= sim_time_step

        print(time_, state)

        episode_over, goal = check_goal(plant, plant_context)

        if time_ >= max_sim or episode_over or shot_time > MAX_TIME_SINCE_SHOT:
            break
    TIME += time_
    print("Goal", goal)
    meshcat.PublishRecording()
    return TIME

In [ ]:
#can keep running this cell to run an episode need to keep track of time
TIME = run_episode(plant, plant_context, TIME)

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

0.1 moving
0.2 passing
0.30000000000000004 collecting
0.4 collecting
0.5 collecting
0.6 collecting
0.7 collecting
0.7999999999999999 collecting
0.8999999999999999 collecting
0.9999999999999999 collecting
1.0999999999999999 collecting
1.2 moving
1.3 moving
1.4000000000000001 moving
1.5000000000000002 moving
1.6000000000000003 moving
1.7000000000000004 moving
1.8000000000000005 moving
1.9000000000000006 moving
2.0000000000000004 moving
2.1000000000000005 moving
2.2000000000000006 moving
2.3000000000000007 moving
2.400000000000001 moving
2.500000000000001 moving
2.600000000000001 moving
2.700000000000001 passing
2.800000000000001 collecting
2.9000000000000012 collecting
3.0000000000000013 collecting
3.1

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443541ce-e170-4459-8340-438ed0837988' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>